## Load data (colab environment)


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
PRODUCTS_PATH = 'drive/MyDrive/data - MoMD'

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import get_json_object, from_json, regexp_replace, col
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Office').getOrCreate()

review_df = spark.read.json(PRODUCTS_PATH + '/Office_Products.jsonl')

In [13]:
meta_schema = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", DoubleType(), True),
    StructField("rating_number", LongType(), True),
    StructField("features", ArrayType(StringType()), True),
    StructField("description", ArrayType(StringType()), True),
    StructField("price", DoubleType(), True),
    StructField("images", ArrayType(MapType(StringType(), StringType())), True),
    StructField("videos", ArrayType(MapType(StringType(), StringType())), True),
    StructField("store", StringType(), True),
    StructField("categories", ArrayType(StringType()), True),

    StructField("details", StringType(), True),

    StructField("parent_asin", StringType(), True),
    StructField("bought_together", DoubleType(), True),
    StructField("subtitle", StringType(), True),
    StructField("author", StringType(), True),
])

raw_meta_df = spark.read.text(PRODUCTS_PATH + "/meta_Office_Products.jsonl")

meta_df = (
    raw_meta_df
    .select(from_json(col("value"), meta_schema).alias("data"))
    .select("data.*")
)

In [14]:
review_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)



## Review Text

### Length

In [15]:
from pyspark.sql.functions import length, col

review_df = review_df.withColumn("review_length", length(col("text"))) \
                    .withColumn("title_length", length(col("title")))


In [16]:
review_df.select('review_length', 'title_length').describe().show()

+-------+------------------+------------------+
|summary|     review_length|      title_length|
+-------+------------------+------------------+
|  count|          12845712|          12845712|
|   mean| 175.3690408129966|21.797346616520752|
| stddev|269.27957946086855|16.346089135924515|
|    min|                 0|                 1|
|    max|             33432|               415|
+-------+------------------+------------------+



In [30]:
quantiles = review_df.approxQuantile(
    col=['review_length', 'title_length'],
    probabilities=[0.25, 0.5, 0.75],
    relativeError=0.001
)

quantiles

[[42.0, 100.0, 208.0], [10.0, 17.0, 28.0]]

In [39]:
review_df.filter(col("text") == col("title")).count()

232364

### Short review

In [49]:
short_title_df = review_df.filter(col("title_length") <= 5).select("title", "text")

In [50]:
short_title_df.show(10, truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title|text                                                                                                                                                                                       |
+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Fun  |The only thing is that you really need a nice smooth paper for these. They are fun to use. I do wish the pen size was a little larger point but overall a fun addition to craft/cardmakers.|
|Fine |As advertised                                                                                                                                                                              |
|Great|Wonderful cus

In [51]:
short_title_df.count()

554708

In [41]:
short_content_df = review_df.filter(col("review_length").between(1, 10)).select("title", "text")

In [42]:
short_title_df.show(10, truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title|text                                                                                                                                                                             |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|.    |👍🏻                                                                                                                                                                             |
|.    |Shipped and arrived when expected, is what its supposed to be, does exactly what it should, no complaints.                                                                       |
|.    |Shipped and arrived when expected, is what its supposed to be, do

In [43]:
short_content_df.count()

595860

In [52]:
(short_content_df.intersect(short_title_df)).count()

25834

### Empty review

In [55]:
empty_review_df = review_df.filter(col("text") == "").select("title", "text", "title_length")

In [56]:
empty_review_df.show(10, truncate=False)

+---------------------------------------------+----+------------+
|title                                        |text|title_length|
+---------------------------------------------+----+------------+
|Four Stars                                   |    |10          |
|Nice case                                    |    |9           |
|I love these                                 |    |12          |
|LOOKS GREAT, DOESN'T LAST                    |    |25          |
|Two Stars                                    |    |9           |
|Five Stars                                   |    |10          |
|Love it! 👌🏼                                |    |11          |
|Read this, but honestly it's luck of the draw|    |45          |
|Five Stars                                   |    |10          |
|Nice!!                                       |    |6           |
+---------------------------------------------+----+------------+
only showing top 10 rows


In [57]:
empty_review_df.filter(col("title_length") == 1).count()

1

## Helpful vote

In [60]:
review_df.select("helpful_vote").describe().show()

+-------+------------------+
|summary|      helpful_vote|
+-------+------------------+
|  count|          12845712|
|   mean|0.8696842962071701|
| stddev| 18.39502015534342|
|    min|                -1|
|    max|             41687|
+-------+------------------+



In [62]:
review_df.filter(col("helpful_vote") < 0).count()

1

In [63]:
review_df.approxQuantile(
    col=['helpful_vote'],
    probabilities=[0.25, 0.5, 0.75],
    relativeError=0.001
)

[[0.0, 0.0, 0.0]]

In [69]:
helpful_review_df = review_df.filter(col("helpful_vote") > 10)

In [70]:
helpful_review_df.count() / review_df.count()

0.012445943050879546

In [71]:
helpful_review_df.groupBy("rating").count().orderBy("rating").show()

+------+-----+
|rating|count|
+------+-----+
|   1.0|30913|
|   2.0|10049|
|   3.0|12750|
|   4.0|20721|
|   5.0|85444|
+------+-----+

